In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

n: the number of points in supp(X)

m: the number of points in supp(\Tilde(X))

In [27]:
n=4

In [28]:
dist=dict()
dist['s']=np.array([0.6,0.4])
dist['x_0']= (1/n)*np.ones(n)
dist['x_1']= np.concatenate(((1/(n-2))*np.ones(n-2),np.zeros(2)),axis=None)
dist['x']=dist['s'][0]*dist['x_0']+dist['s'][1]*dist['x_1']
dist['t_x']= (1/n)*np.ones(n)

In [29]:
v=(dist['x_0']-dist['x_1'])/dist['x']

In [39]:
e=0.01

sum(gamma) == gamma.T \mathbb{1}

sum(gamma.T) == gamma \mathbb{1}

In [41]:
xi=xi_generate([*range(n)],e)

In [46]:
gamma=xi

In [50]:
dist['x']

array([0.35, 0.35, 0.15, 0.15])

In [73]:
dist['t_x']

array([0.25, 0.25, 0.25, 0.25])

In [71]:
gamma

array([[  0., 100., 200., 300.],
       [100.,   0., 100., 200.],
       [200., 100.,   0., 100.],
       [300., 200., 100.,   0.]])

In [70]:
new_gamma

array([[0.        , 0.05833333, 0.11666667, 0.175     ],
       [0.0875    , 0.        , 0.0875    , 0.175     ],
       [0.075     , 0.0375    , 0.        , 0.0375    ],
       [0.075     , 0.05      , 0.025     , 0.        ]])

In [72]:
newnew_gamma

array([[0.        , 0.1       , 0.12727273, 0.11290323],
       [0.09210526, 0.        , 0.09545455, 0.11290323],
       [0.07894737, 0.06428571, 0.        , 0.02419355],
       [0.07894737, 0.08571429, 0.02727273, 0.        ]])

In [86]:
# simplist
gamma=xi
for repeat in range(100):
    new_gamma=np.diag(dist['x']/sum(gamma.T))@gamma
    new2_gamma=new_gamma@np.diag(dist['t_x']/sum(new_gamma))
    gamma=new2_gamma

In [87]:
theta=abs(gamma.T @ v)*(1-0.3)

In [ ]:
gamma=xi
for repeat in range(100):
    new_gamma=np.diag(dist['x']/sum(gamma.T))@gamma
    new2_gamma=new_gamma@np.diag(dist['t_x']/sum(new_gamma))
    

In [36]:
def xi_generate(x_range,e):
    bin=len(x_range)
    C=np.random.random((bin,bin))
    for i in range(bin):
        for j in range(bin):
            C[i,j]=abs(x_range[i]-x_range[j])/e
    return C

In [ ]:

def algorithms(reg,m,n,g,f,C):
    K=np.exp(-C/C.max()/reg)
    interations=100
    trial=10 #check the optimiality every trial. 
    u=np.ones((n,1))
    u_record=np.ones((n,1))
    for tri in range(trial):
        if sum(abs(u_record-u))<10**(-10):
            for i in range(1,interations):
                v=g/np.dot(K.T,u)
                u=f/np.dot(K,v)
            u_record=u
        else:
            break
    return np.dot(np.diag(u.reshape((1,-1))[0]),np.dot(K,np.diag(v.reshape((1,-1))[0])))

def assess(m,n,g,f,C,output):
    print('sum of violation of f:',sum(abs(np.sum(output,1)-f.reshape(n))))
    print('sum of violation of g:',sum(abs(np.sum(output,0)-g.reshape(m))))
    print('total cost:',sum(sum(output*C)))
    print('entropy:',sum(sum(-output*np.log(output+0.1**3))))
    print('============================================')

def plots(x_range,g,f,output):
    fig = plt.figure(figsize=(3,3))
    gs = fig.add_gridspec(2, 2, width_ratios=(4,1), height_ratios=(1,4),left=0.1,right=0.9,bottom=0.1, top=0.9,wspace=0,hspace=0)
    # Create the Axes.
    ax = fig.add_subplot(gs[1, 0])
    ax.pcolormesh(x_range, x_range, output, cmap='Blues')
    ax.set_xlabel(r'$\mathbb{X}$',fontsize=14)
    ax.set_ylabel(r'$\tilde{\mathbb{X}}$',rotation='horizontal',fontsize=14)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax) 
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
    #ax_histx.set_title(r'$Pr[x]$',rotation='horizontal')
    #ax_histy.set_title(r'$Pr[\tilde{x}]$')
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histx.tick_params(axis="y", labelleft=False)
    ax_histy.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    ax_histx.plot(x_range,g,color='tab:blue')
    ax_histy.plot(f,x_range,color='tab:green') 
    return fig

def OT(reg,x_range,g,f):
    m=len(g)
    n=len(f)
    C=c_generate(x_range)
    output=algorithms(reg,m,n,g.reshape(m,-1),f.reshape(n,-1),C)
    assess(m,n,g,f,C,output)
    plots(x_range,g,f,output)
    return outputa